# Shipment Tracker API

In [1]:
import pandas as pd

# Set display option to show all columns and max column width
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### File Paths and Backup

In [2]:
from functions0_basics import (get_files_path,
                               save_backup,
                               shipments_not_delivered,
                               get_API_details, save_to_excel, convert_urls_to_links)

# Get files paths and store them into variables (chromedriver if needed)
excel_path, report_path, chromedriver_path = get_files_path("Albert")

# Create a backup of original Excel file 
save_backup(excel_path)

Backup saved to: ./Shipment Data/BackUp - Daily Report Shipments of Spare Parts.xlsx 27-12-2023 11_59_48.xlsx
Deleted old backup: ./Shipment Data/BackUp - Daily Report Shipments of Spare Parts.xlsx 26-12-2023 20_57_47.xlsx


In [3]:
# Get data for all carriers != 'Delivered'
shipments_not_delivered = shipments_not_delivered(excel_path)

**150 shipments NOT DELIVERED in your file**

Status,Carrier,In Transit,Exception,Totals
0,DHL,16,0,16
1,NACEX,2,1,3
2,SEUR,1,1,2
3,TNT,127,2,129


### TNT

In [4]:
# TNT: API Requests
from functions1_TNT_requests import make_tnt_requests
tnt_results, len_shipm_numbers = make_tnt_requests(shipments_not_delivered)

In [5]:
# TNT: Extract, process and create dataframe
from functions2_TNT_dataframe import tnt_to_dataframe
tnt_df = tnt_to_dataframe(tnt_results, shipments_not_delivered, len_shipm_numbers, report_path)
# Saved df:
#tnt_df

# Linkable df:
# tnt_df_linkable = convert_urls_to_links(tnt_df)

Successfully retrieved all TNT shipments data in attempt 1.
 


New report file saved at: Track Reports/TNT - Track Report 27-12-2023 11_59_53.xlsx

### DHL

In [6]:
# DHL: API Requests
from functions1_DHL_requests import make_dhl_requests
all_dhl_results, max_dhl_shipm = make_dhl_requests(shipments_not_delivered)

Rate limited for URL https://api-eu.dhl.com/track/shipments?trackingNumber=1465773864. Retrying each shipment individually.


In [7]:
# DHL: Extract, process and create dataframe
from functions2_DHL_dataframe import dhl_to_dataframe
dhl_df = dhl_to_dataframe(all_dhl_results, shipments_not_delivered, max_dhl_shipm, report_path)
# Saved df:
#dhl_df

# Linkable df:
# dhl_df_linkable = convert_urls_to_links(dhl_df)

Successfully retrieved all DHL shipments data in attempt 1.
 


New report file saved at: Track Reports/DHL - Track Report 27-12-2023 12_00_01.xlsx

In [8]:
dhl_df.columns

Index(['Carrier', 'Client Reference', 'Shipment Num.', 'Service',
       'Origin Date', 'From (City)', 'From (Country)', 'To (City)',
       'To (Country)', 'Num. of Pieces', 'Processing Days', 'Carrier Status',
       'Last Update (Date)', 'Last Update (Hour)', 'Last Location (City)',
       'Last Location (Country)', 'Last Action', 'Exception Notification',
       'Shipment URL', 'POD Status', 'POD Link', 'POD Signature Link',
       'Remark', 'Next Steps', 'Estimated Date Delivery',
       'Estimated Time Delivery'],
      dtype='object')

In [9]:
tnt_df.columns

Index(['Carrier', 'Client Reference', 'Shipment Num.', 'Origin Date',
       'From (City)', 'From (Country)', 'To (City)', 'To (Country)',
       'Num. of Pieces', 'Processing Days', 'Summary Code', 'Signatory',
       'Carrier Code Status', 'Last Update (Date)', 'Last Update (Hour)',
       'Last Location', 'Last Action', 'Exception Notification', 'URL'],
      dtype='object')

In [10]:
tnt_df.head(4)

,Carrier,Client Reference,Shipment Num.,Origin Date,From (City),From (Country),To (City),To (Country),Num. of Pieces,Processing Days,Summary Code,Signatory,Carrier Code Status,Last Update (Date),Last Update (Hour),Last Location,Last Action,Exception Notification,URL
0,TNT,DSD/140232,663782937,24-11-2023,Barcelona,Spain,Krnjesevci,Serbia,1,23,In Transit,,HW,24-11-2023,21:51,Barcelona,Shipment arrived at TNT location,,https://www.tnt.com/express/en_gc/site/shipping-tools/track.html?searchType=con&cons=663782937
1,TNT,DSD/142142,663787660,28-11-2023,Barcelona,Spain,Shkoder,Albania,1,13,Delivered,L HABIBI,OK,15-12-2023,16:30,Albania Others,Shipment delivered in good condition,,https://www.tnt.com/express/en_gc/site/shipping-tools/track.html?searchType=con&cons=663787660
2,TNT,DSD/146902,663806545,07-12-2023,Barcelona,Spain,Lipari,Italy,1,7,Delivered,MERLINO,OK,18-12-2023,20:02,Messina,Shipment delivered in good condition,,https://www.tnt.com/express/en_gc/site/shipping-tools/track.html?searchType=con&cons=663806545
3,TNT,DSD/146772,663805920,07-12-2023,Barcelona,Spain,Legnago,Italy,2,5,Delivered,Bisco,OK,14-12-2023,15:11,Verona,Shipment delivered in good condition,,https://www.tnt.com/express/en_gc/site/shipping-tools/track.html?searchType=con&cons=663805920
